In [3]:
import tensorflow as tf

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2021-11-16 03:31:52.440060: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-16 03:31:52.850329: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 22298 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1633056768880663514
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 23381475328
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3435841969169439158
 physical_device_desc: "device: 0, name: GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [5]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline

In [6]:
import sys
#sys.path.append('/home/dwkim/project/labranchor/')
#from labranchor.create_fasta import Genome
from genome import Genome
genome = Genome('/home/dwkim/gencode_v19_raw_data/labranchor.gencode_v19.fa')

In [7]:
introns = {}
with open('/home/dwkim/anno/introns_to_mercer.tsv') as fp:
    for line in fp:
        chrom, start, end, _, pos, strand, _, bp = line.split('\t')[:8]
        bp, start, end = int(bp), int(start), int(end)
        
        three = end if strand == '+' else start  
        key = (chrom, three, strand)
        
        if not 5 < abs(bp - three) < 60:
            bp = -1
        
        if key not in introns: introns[key] = []
        assert bp not in introns[key], bp
        if bp != -1: introns[key] += [bp]


In [8]:
# Used to compare to branchpointer
c = 0
with open('./test.tsv', 'w') as fp:
    ID = 0
    fp.write('\t'.join(['id', 'chromsome', 'start', 'end', 'strand']) + '\n')
    for (chrom, three, strand), bp in introns.items():
        if not bp or chrom != 'chr1':
            continue
        if strand == '+':
            begin, end = str(three - 70), str(three)
        else:
            c += 1
            begin, end = str(three+1), str(three + 70+1)
        
        fp.write('\t'.join(["chr1_{}_{}".format(three, strand),
                            'chr1', begin, end, strand]) + '\n')
print(c)

2145


In [9]:
known   = {key: value for key, value in introns.items() if value}
missing = {key: value for key, value in introns.items() if not value}
print(len(known))
print(len(missing))

37110
169182


In [10]:
L = 70
bases = ['A', 'C', 'G', 'T']

def onehot(seq):
    X = np.zeros((len(seq), len(bases)))
    for i, char in enumerate(seq):
        X[i, bases.index(char)] = 1
    return X

count =0 
X, y, chroms = [], [], []
for intron, bps in known.items():
    count+=1
    if count % 1000 == 0:
        print(count)
    chrom, three, strand = intron
    if strand == '+':
        begin, stop = three - L, three
    else:
        begin, stop = three, three + L
    
    # Get features
    seq = genome.get_seq(chrom, begin, stop, strand)
    #print(seq)
    if 'N' in seq: seq = seq.replace('N', 'A')
    
    X += [onehot(seq).reshape(1, 70, 4)]

    # Make target
    _y = np.zeros((stop - begin,))
    for bp in bps:
        if strand == '+':
            bp = L + bp - three
        else:
            bp = L - bp + three - 1
        _y[bp] = 1
    y += [_y]
    
    chroms += [chrom]

X, y = np.vstack(X), np.vstack(y)
print(X.shape, y.shape, len(chroms))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
(37110, 70, 4) (37110, 70) 37110


In [11]:
test = np.array(list(map(lambda x: x == 'chr1', chroms)))
valid = np.array(list(map(lambda x: x in ('chr2', 'chr3', 'chr4', 'chr5'),
                     chroms)))

train = np.array([not (t or v) for t, v in zip(test, valid)])
print(sum(test),  sum(valid), sum(train))

X_train, X_valid, X_test = X[train], X[valid], X[test]
y_train, y_valid, y_test = y[train], y[valid], y[test]
print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

4306 7093 25711
(25711, 70, 4) (7093, 70, 4) (4306, 70, 4)
(25711, 70) (7093, 70) (4306, 70)


In [25]:
#np.save('./X_train.npy', X_train)
#np.save('./X_valid.npy', X_valid)
#np.save('./X_test.npy', X_test)

(25711, 70, 4)

In [26]:
def matching(preds, true):
    total, match = 0, 0
    for p, t in zip(preds, true):
        if np.argmax(p) in np.nonzero(t)[0]:
            match += 1
        total += 1
    return match, total

In [33]:
K = 3
counts = np.zeros((2*K+1, 4))
for target, seq in zip(y_train, X_train):
    for bp in np.nonzero(target)[0]:
        if 0 > bp-K or bp+K+1 > seq.shape[0]: continue
        counts = counts + seq[bp-K: bp+K+1]
print(counts.T)

[[ 8658.  8037.  8984. 20929.  9468.  9994.  9281.]
 [12046.  8055. 11683.  6820. 11738. 11016. 10508.]
 [ 8100.  7269.  8965.  5728.  8041.  7817.  8519.]
 [10247. 15690.  9419.  5574.  9804. 10224. 10743.]]


In [37]:
class ModelTrainer:
    def __init__(self, model):
        self.model = model
        self.train_auc = []
        self.train_match = []
        self.valid_auc = []
        self.valid_match = []
        
    def train(self, X_train, X_valid,
              y_train, y_valid, PATIENCE = 15, EPOCHS = 80):
        print(model.summary())
        for i in range(EPOCHS):
            model.fit(X_train, y_train, epochs = 1, verbose = 0, batch_size = 8)
            self._evaluate(X_train, X_valid, y_train, y_valid)
            if (i > PATIENCE
                and max(self.valid_match[-PATIENCE:])
                < max(self.valid_match)):
                break
            print(i, self.valid_match[-1], self.train_match[-1])
        self._plot_scores()
        print(max(self.valid_match), max(self.valid_auc))
        print('saved model')
        model.save('./2layer_gpu.h5')
    
    def predict(X):
        return self.model.predict(X)
                
    def _evaluate(self, X_train, X_valid, y_train, y_valid):
        valid_preds = self.model.predict(X_valid, verbose=0)
        train_preds = self.model.predict(X_train, verbose=0)
        self.valid_match += [matching(valid_preds, y_valid)[0]
                             / float(y_valid.shape[0])]
        self.train_match += [matching(train_preds, y_train)[0]
                             / float(y_train.shape[0])]
        self.valid_auc += [metrics.roc_auc_score(y_valid.flatten(),
                                                 valid_preds.flatten())]
        self.train_auc += [metrics.roc_auc_score(y_train.flatten(),
                                                 train_preds.flatten())]
    
    def _plot_scores(self):
        plt.plot(self.valid_match, label = 'Validation')
        plt.plot(self.train_match, label = 'Training')
        plt.ylabel('Match')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()
        
        plt.plot(self.valid_auc, label = 'Validation')
        plt.plot(self.train_auc, label = 'Training')
        plt.ylabel('auROC')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()

In [38]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, LSTM, Dropout, Reshape
from tensorflow.keras.layers import Bidirectional, TimeDistributed
from tensorflow.keras import regularizers, optimizers

In [ ]:
model = Sequential()

model.add(Bidirectional(LSTM(32, return_sequences = True,
                             dropout = 0.15, recurrent_dropout = 0.05),
                            input_shape = (L, 4)))
model.add(Bidirectional(LSTM(32, return_sequences = True,
                             dropout = 0.15, recurrent_dropout = 0.05)))

model.add(TimeDistributed(Dense(1, activation = 'sigmoid')))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(learning_rate=0.001,
                                        beta_1=0.9,
                                        beta_2=0.999,
                                        epsilon=1e-08,
                                        decay=0.0))

ModelTrainer(model).train(X_train[:, :, :4], X_valid[:, :, :4],
                          y_train.reshape(-1, 70, 1), y_valid.reshape(-1, 70, 1))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_8 (Bidirectio  (None, 70, 64)           9472      
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 70, 64)           24832     
 nal)                                                            
                                                                 
 time_distributed_4 (TimeDis  (None, 70, 1)            65        
 tributed)                                                       
                                                                 
Total params: 34,369
Trainable params: 34,369
Non-trainable params: 0
_________________________________________________________________
None
0 0.3383617651205414 0.35082260511065305
1 0.3545749330325673 0.36894714324608147
2 0.35809953475257295 0.37567

In [ ]:
model.save('./2layer_gpu.h5')